# 实验报告
### 计21 展子航 2022010745

## 引言（碎碎念）
虽然中途发了四天烧，但是我的大作业总算还是按时完成了。其实本次收获最大的部分是我查询资料和快速学习新库的能力，同时也增加了不少码力（毕竟还是写了很多的），只希望大模型demo能对我好一些hh

说来其实爬虫对我算很温柔的，因为我曾经有自学过前端三巨头并轻度使用过一些网页源代码查看工具，所以我上手是比较快的，再加上有科协的爬虫课程（我看完了2个小时）的辅助，起步要容易很多，当时身体有些不适也被我略过去了。

到后来入门django的时候我就开始生病了（上周三上课时就不舒服，之后高烧了一天，又断断续续烧了三天，周日才算彻底退烧），其中四天基本是躺着度过的，可谓是非常痛苦，感觉自己能做完作业已经非常不错了555。

到后来退烧以后就开始狂赶进度，有一种项目加急上线的美感，对python的基本工具已经信手拈来了（笑），最终赶上了周一的验收，除了搜索引擎有不少提升空间以外，别的已经尽力能做到达标了。

这个故事告诉我们需要学会爱护自己的身体，不要在之前写得入魔了天天熬夜到三点，这样子后面一场病使得前面赶的进度全部水漂~
## 环境准备
anaconda下的base环境以及自建的mach虚拟环境，主要使用mach环境来进行开发。其中爬虫和数据分析部分更多是在vscode下完成（因为pycharm如果要看ipynb就要专业版软件）而web部分更多是在pycharm下完成（变量和文件更名方便、会自动重构整个代码的引用，非常省时间）

## 作业介绍
### 爬虫部分
* 爬取的网站：新浪滚动新闻；
* 所使用的方法：寻找新浪后端供给新闻链接条目的api，最终发现是https://feed.mix.sina.com.cn/api/roll，然后利用JSON解析器直接找到内部存留的所有链接，存储在一个txt文件中；
* 爬取的url量：从4月1日到8月1日数据量在8776条：

In [ ]:
import requests
import json
from bs4 import BeautifulSoup as bs
from time import sleep as slp
from typing import List
import random
import pandas as pd
import re
from datetime import datetime
import os

In [ ]:
config = {"User-Agent": 
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54",
            } #伪装成浏览器访问

def get_urls(urlx, start_page, dt) -> List[str]: #写一个爬其中一页的函数
    dt_c = datetime.strptime(dt, '%Y-%m-%d')
    dt_stamp = int(dt_c.timestamp())
    #print(dt_stamp)
    st_stamp = dt_stamp + 86400
    news_list = []
    try:
        repo = requests.get(url = urlx, params = {"pageid": 153, "lid": 2515, "num": 50, "date": dt, "page": start_page, "stime": st_stamp, "ctime": st_stamp, "etime": dt_stamp}, headers = config)
        repo.encoding = "UTF-8"
        tex = repo.json() # 解析
        # 然后直接找链接
        leng = len(tex["result"]["data"])
        news_list += [ tex["result"]["data"][index]["url"] for index in range(leng)] 
    except:
        news_list.append["cannot reach"]
    return news_list



dates = pd.date_range(start = "2023-07-01", periods = 30).strftime("%Y-%m-%d").tolist()
# dates这个数据是自行修改的
dates = dates[::-1]
for dt in dates:
    for page in range(1, 10):
        lst = get_urls(f"https://feed.mix.sina.com.cn/api/roll/get", page, dt)
        with open(r'./news_links_lasty.txt', 'a', encoding = "utf-8") as f:
            for item in lst:
                if item != "cannot reach":
                    f.write(f"{item}\n")
            f.close()

之后处理新闻要素，本质上是使用BeautifulSoup4用lxml解析器解析并且处理html文本，然后将所有的图片中缺失http前缀的全部补上方便下载，之后提取所有我想要的要素，如果实在提取不到就用正则表达式匹配原文本。由于我发现新浪的新闻有三种html格式，其中第一种和第二种高度相像，第三种要更特别一些，因此先运行以下代码，将输出的无法使用的url留存：

In [ ]:
#第二步：处理新闻要素
from PIL import Image
import traceback
import multiprocessing
config = {"User-Agent": 
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54",
        }

def get_html(url):
    slp(random.uniform(0.3, 0.5))
    repo = requests.get(url = url, headers = config)
    repo.encoding = 'utf-8'
    html = repo.text
    # print(url)
    return html

def parse_html(html, url):
    try:
        soup = bs(html, "lxml") # 先解析网页
        
        # 新浪有个贼坑的专栏叫"创事记"，如果作者是他的话发行日期和文章截取得特判
        author = re.findall('<meta name="mediaid" content="(.*?)"', html)[0] # 作者
        if author == "创事记":
            publish_time = soup.find("span", id = "pub_date").text # 发行日期
            publish_time = publish_time.strip()
            date_str = publish_time
            date_obj = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
        else:
            publish_time = soup.find("span", class_ = "date").text # 发行日期
            date_str = publish_time
            date_obj = datetime.strptime(date_str, "%Y年%m月%d日 %H:%M")
        
        new_date_str = date_obj.strftime("%Y-%m-%d")
        publish_time = date_obj.strftime("%Y-%m-%d %H:%M")
        dir_name = re.findall('[0-9]\/(.+?)\.shtml', url)[0] # 给网站起个名字
        dir_name = './websites_ten/' + new_date_str + dir_name
        # print(dir_name)
        try:
            if not os.path.exists(dir_name):
                os.mkdir(dir_name) # 一个网站一个文件夹
                
        except:
            print("website already create")
        
        dir_img = dir_name + '/images'
        
        try:
            if not os.path.exists(dir_img):
                os.mkdir(dir_img) # 一个网站一个文件夹
                
        except:
            print("image site already create")
        
        title = re.findall('<meta property="og:title" content="(.*?)"', html)[0] # 新闻标题
        
        description = re.findall('<meta name="description" content="(.*?)"', html) # 摘要
        if len(description) > 0:
            description = description[0]
        keywords_source = soup.find("div", class_ = "keywords") # 关键词-1
        keywords = []
        if keywords_source is not None:
            keywords = keywords_source.find_all("a") # 关键词-2
            keywords = [item.text for item in keywords] # 关键词-3
        # print(keywords)
        passage_naked = "" # 整个文章
        passage_origin = ""
        passage = soup.select(".article p") # 直接元素选择器
        if passage == []:
            passage = soup.select("#artibody p") # 直接元素选择器
        for content in passage:
            passage_naked += (content.text + "\n")
            passage_origin += content.text
        # print(passage_naked)
        dic_ = {"url": url, "title": title, "author": author, "description": description, "publish_time": publish_time, "keywords": keywords}
        json_dic = json.dumps(dic_, indent = 2) # 将以上文件通过dict转化为json utf-8
        # print(json_dic)
        json_dir = dir_name + '/config.json' # json文件的地址
        article_dir = dir_name + '/article.txt' # 文章内容地址
        article_without = dir_name + '/article_origin.txt' # 文章内容地址
        with open(json_dir, 'w') as f:
            f.write(json_dic) # 写入config.json文件
            f.close()
        
        with open(article_dir, 'w') as f:
            f.write(passage_naked) # 写入article.txt文件
            f.close()
            
        with open(article_without, 'w') as f:
            f.write(passage_origin) # 写入article.txt文件
            f.close()
            
        img_links = []
        imgs = soup.find_all("div", class_ = "img_wrapper")
        for item in imgs:
            img_link_origin = item.find("img")
            #print(img_link_origin)
            try:
                img_link_origin = img_link_origin["src"]
                if not(re.match("^https:", img_link_origin) or re.match("^http:", img_link_origin)):
                    img_link_origin = "https:" + img_link_origin
                if img_link_origin is not None:
                    img_links.append(img_link_origin)
            except:
                pass
        # print(img_links)
        index = 0
        img_dic = {"empty": 1} #图片对应的id和链接
        
        datax = {}
        for one in img_links:
            img_dic["empty"] = 0;
            index += 1
            name_tmp = one.split('/')[-1]
            datax[f"{index}"] = [name_tmp, one];
            img_data = requests.get(url = one).content
            with open(os.path.join(dir_img, name_tmp), 'wb') as f:
                f.write(img_data)
                f.close()
        
        img_dic['data'] = datax
        img_json_dic = json.dumps(img_dic, indent = 2)
        json_dir = dir_img + '/img_config.json' # json文件的地址
        with open(json_dir, 'w') as f:
            f.write(img_json_dic) # 写入img_config.json文件
            f.close()
    except Exception as ex:
        print(url)

    
    
def main():
    # with multiprocessing.Pool(processes=8) as pool:
    with open(r"./news_links_lasty.txt", "r", encoding = "utf-8") as f:
        urlsx = f.readlines()
    urlsx = [ url.strip() for url in urlsx ]
    for url in urlsx:
        parse_html(get_html(url), url)
        
        
if __name__ == "__main__":  
   main() 

之后只要让所有报错的url去执行第二个就可以了~

In [ ]:
#以下是专门为新浪的苹果新闻定制的小爬虫
from PIL import Image
config = {"User-Agent": 
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.54",
        }
with open(r"./pa2.txt", "r", encoding = "utf-8") as f:
    urlsx = f.readlines()
# print(urlsx)
urlsx = [ url.strip() for url in urlsx ]
# print(urlsx)
for url in urlsx:
    try:
        slp(random.uniform(0.3, 0.5))
        repo = requests.get(url = url, headers = config)
        repo.encoding = 'utf-8'
        html = repo.text
        # print(html)
        soup = bs(html, "lxml") # 先解析网页
        
        #关键词模块重做：根据观察，放在meta里面了
        keywords = re.findall(r'<meta name="keywords" content="(.*?)"' , html)[0]
        keywords = keywords.split(',')
        # print(keywords)
        
        title = re.findall('<meta property="og:title" content="(.*?)"', html)[0] # 新闻标题
        author = re.findall('<meta property="article:author" content="(.*?)"', html)[0] # 作者
        # print(author)
        
        #发行日期模块重做
        publish_time = re.findall(r'<span id="pub_date">(.*?)</span>', html)[0] # 发行日期
        date_obj = datetime.strptime(publish_time, "%Y年%m月%d日%H:%M")
        new_date_str = date_obj.strftime("%Y-%m-%d")
        publish_time = date_obj.strftime("%Y-%m-%d %H:%M")
        # print(new_date_str)
        dir_name = re.findall('[0-9]\/(.+?)\.shtml', url)[0] # 给网站起个名字
        dir_name = './websites/' + new_date_str + dir_name
        # print(dir_name)
        try:
            if not os.path.exists(dir_name):
                os.mkdir(dir_name) # 一个网站一个文件夹
                dir_img = dir_name + '/images'
                os.mkdir(dir_img) # 文件夹中有一个图片夹
        except:
            print("website already create")

        description = re.findall('<meta name="description" content="(.*?)"', html) # 摘要
        if len(description) > 0:
            description = description[0]
        # print(keywords)
        passage_naked = "" # 整个文章
        passage = soup.select("#artibody p") # 直接元素选择器
        for content in passage:
            passage_naked += (content.text + "\n")
        # print(passage_naked)
        dic_ = {"url": url, "title": title, "author": author, "description": description, "publish_time": publish_time, "keywords": keywords}
        json_dic = json.dumps(dic_, indent = 2) # 将以上文件通过dict转化为json utf-8
        # print(json_dic)
        json_dir = dir_name + '/config.json' # json文件的地址
        article_dir = dir_name + '/article.txt' # 文章内容地址
        with open(json_dir, 'w') as f:
            f.write(json_dic) # 写入config.json文件
            f.close()
        
        with open(article_dir, 'w') as f:
            f.write(passage_naked) # 写入article.txt文件
            f.close()
            
        img_links = []
        imgs = soup.find_all("div", class_ = "img_wrapper")
        for item in imgs:
            img_link_origin = item.find("img")["src"]
            if not re.match("^https:", img_link_origin):
                img_link_origin = "https:" + img_link_origin
            img_links.append(img_link_origin) 
        # print(img_links)
        index = 0
        img_dic = {"empty": 1} #图片对应的id和链接
        
        for one in img_links:
            img_dic["empty"] = 0;
            index += 1
            name_tmp = one.split('/')[-1]
            img_dic[f"{index}"] = name_tmp;
            img_data = requests.get(url = one).content
            with open(os.path.join(dir_img, name_tmp), 'wb') as f:
                f.write(img_data)
                f.close()
                
        img_json_dic = json.dumps(img_dic, indent = 2)
        json_dir = dir_img + '/img_config.json' # json文件的地址
        with open(json_dir, 'w') as f:
            f.write(img_json_dic) # 写入img_config.json文件
            f.close()
    except Exception as ex:
        print(url)
        print(repo)

至此，我们得到了全部8776篇新闻的概述（json保存）、图片（全部下载）、文本。
### 网站部分
* 主要结构：django的基本结构，以及一个专门用来导入新闻的脚本，下面是脚本：

In [ ]:
import json
import os
from datetime import datetime, timedelta
import traceback
import numpy as np
from news.models import News, Category

path = r'D:\involuntary\works\missing-semester-of-python\part1\websites'
files = os.listdir(path)

# News.objects.all()

for file in files:
    try:
        all_path = path + '\\' + file
        config_path = all_path + r'\config.json'
        img_path = all_path + r'\images\img_config.json'
        article_path = all_path + r'\article.txt'
        content = {}
        contenTxt = ''
        
        # 导入所有文件
        with open(config_path, 'r') as f:
            json_config = f.read() 
        xconfig = json.loads(json_config)
        with open(article_path, 'r', encoding='utf-8') as g:
            pas = g.readlines()
            index = 0
            for item in pas:
                content[f'{index}'] = item.rstrip()
                contenTxt += item.rstrip()
                index += 1
        with open(img_path, 'r') as h:
            img_config = h.read()
            img_bababa = json.loads(img_config)
        short = content['0'][:50] if content['0'] else "无预览"
        category = ""
        now_time = datetime.now()
        pub_time = datetime.strptime(xconfig['publish_time'], '%Y-%m-%d %H:%M')
        if pub_time + timedelta(days=60) > now_time:
            category = "两个月以内"
        elif pub_time + timedelta(days=90) > now_time:
            category = "三个月以内"
        elif pub_time + timedelta(days=120) > now_time:
            category = "四个月以内"
        else:
            category = "其他"
        xx = f'{file}\\images\\'
        whiw = [(xx + f'{img_bababa["data"][key][0]}') for key in img_bababa['data'].keys()]
        new_news = News(config=json_config,
                        pub_time=pub_time,
                        read_num=np.random.randint(low=10, high=100000),
                        author=xconfig['author'],
                        origin=xconfig['url'],
                        title=xconfig['title'],
                        description=xconfig['description'],
                        category_title=category,
                        content=content,
                        short_arg=short + "......",
                        img_config=img_config,
                        comment_num=0,
                        contenTxt=contenTxt,
                        category=Category.objects.get(title=category),
                        image_dir=whiw,
                        )
        new_news.save()
    except:
        print(traceback.print_exc())

* models部分：我使用了Category、News和Comments三个模型，分别进行Cat -> N与N -> Com的外键关联，使得一个分类可以直接管理多条新闻、一条新闻可以直接管理多个评论，同时利用数据库的默认排序，将默认值改为了按发布时间先后，这样子就在数据输入阶段他就会自动按照时间排序了：

In [ ]:
from django.db import models


# Create your models here.

class Category(models.Model):
    """
    a certain category
    """
    title = models.CharField(max_length=200)
    app_label = 'news'

    def __str__(self):
        return self.title

    class Meta:
        app_label = 'news'


class News(models.Model):
    """
    a certain news
    """
    config = models.JSONField()  # 真实的json
    # 有一些变量其实是可以不用的(毕竟我都有json了) 我还是给他弄上了
    # 毕竟这样的调用要正常不少
    pub_time = models.DateTimeField()  # 发布时间
    short_arg = models.TextField()
    category_title = models.CharField(max_length=200, null=True)
    contenTxt = models.TextField(null=True)
    read_num = models.IntegerField()  # 热 度
    author = models.CharField(max_length=200)  # 作者
    origin = models.URLField()  # 原新闻链接
    title = models.CharField(max_length=200)  # 标题
    description = models.TextField()  # 摘要
    content = models.JSONField()  # 内容
    img_config = models.JSONField()
    comment_num = models.IntegerField()
    image_dir = models.JSONField(null=True)

    def __str__(self):
        return self.title

    # 关联外键
    category = models.ForeignKey(
        'Category',
        on_delete=models.SET_NULL,
        null=True,
    )

    class Meta:
        # News.objects提出数据时按照列表指定的字段排序
        ordering = ['-pub_time']


class Comment(models.Model):
    """
    a certain comment
    其实没什么大的要求，有内容有时间就很足够了
    """
    # author = models.CharField(max_length=200)  # 作者
    content = models.TextField()
    pub_time = models.DateTimeField(auto_now_add=True)  # 这样就该是多少就是多少了

    # 关联外键
    news = models.ForeignKey(
        'News',
        on_delete=models.CASCADE,  # 跟随删除
        null=False,
    )

    class Meta:  # 画瓢
        ordering = ['-pub_time']

* 所有的分页部分我都使用了django自带的分页器进行分页，以下是一个示例：

In [ ]:
def news_list(request):
    # news/list/?page=114514   params = {"page": 114514} 模拟新浪的处理方式
    all_news = News.objects.all()
    paginator = Paginator(all_news, 10)
    page_number = int(request.GET.get('page', 1))  # 自动填充一个param，如果有page那就是page，没有就默认1
    if page_number > paginator.num_pages:
        page_number = paginator.num_pages
    elif page_number < 1:
        page_number = 1
    page = paginator.page(page_number)
    if page_number < 6:  # 没有别的工具，只能手动操作力
        if paginator.num_pages <= 10:
            page_range = range(1, paginator.num_pages + 1)
        else:
            page_range = range(1, 11)
    elif (page_number >= 6) and (page_number <= paginator.num_pages - 5):
        page_range = range(page_number - 5, page_number + 5)
    else:
        page_range = range(paginator.num_pages - 9, paginator.num_pages + 1)

    return render(request, 'list.html', {'page': page,
                                         'range': page_range,
                                         'current_num': page.number,
                                         'news': page.object_list,
                                         'num_pages': paginator.num_pages,
                                         'has_previous': page.has_previous(),
                                         'has_next': page.has_next()})


* 在搜索引擎部分，为了支持能够分词搜索（即我如果搜“显卡的电脑”，能够出现所有带“显卡”和“电脑”全部二者的新闻的搜索结果），我使用了jieba库对搜索文本进行分词并且结合内置数据库进行搜索，搜索时间基本能控制在0.4s左右：

In [ ]:
if query:
    x_blink = Q(title__icontains=query) | Q(contenTxt__icontains=query)
    found_news_blink = None
    sera_list = [word for word in jieba.cut_for_search(query) if word not in stop_words]
    for item in sera_list:
        if found_news_blink is None:
            found_news_blink = (Q(title__icontains=item) | Q(contenTxt__icontains=item))
        else:
            found_news_blink &= (Q(title__icontains=item) | Q(contenTxt__icontains=item))
    found_news = all.filter(x_blink | found_news_blink)

* 而所有的刷新功能和翻页功能我都使用了JavaScript来进行：
```javascript
function shift(page) {
        let url = "/news/search/";
        url += "?page=" + page + "&query=" + "{{query}}" + "&sort=" + {{sorting}};
        {% for item in month_list %}
        url += ("&month=" + {{item}})
        {% endfor %}
        window.location.href = url;
    }

function refresh() {
        window.location.reload("news_random");
    }
```

### 数据分析
使用了jieba分词、pandas存储分词后的数据（键值对形式）、wordcloud绘制词云和matplotlib绘制柱状图，详见data_ana.ipynb数据分析报告
### 所有项目共耗时
* 爬虫及其优化：2天，共计12小时左右
* web框架：6天，共计25小时左右（因为生病，每天的工作时间都减少）
* 数据分析：1天半，共计5小时左右